## <span style=color:blue>This notebook builds the function soy_is_here(year,lat,lon), which produces "True" if the 100m x 100m area around lon-lat was a soybean field in the given year.  (At least, according to the data of USDA NASS.)  </span>

## <span style=color:blue>Then we import the dictionary with the lon-lat sequences for each county, and for each year find the first 20 that are in soybean fields, and right 

<span style=color:blue>First step is to create a function that tests, given a year-lon-lat triple, whether there was a soy field at lat-lon during the given year.  This is based on checking files downloaded from https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php.  To understand the meaning of the pixel values, please see https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/meta.php and the files in there, e.g., https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_ia22.htm.  Among other things, you will see that the value of '5' corresponds to soybean fields </span>

### <span style=color:blue> Before using this Jupyter Notebook file </span>
#### <span style=color:blue> it would help if you ran the 'NDVI_02a_Finding-wheat-lon-lats.py' python script. </span>

In [1]:
import json

working_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
dict_seq_file = 'year_state_county_winterwheat_seq.json'


with open(working_dir+dict_seq_file, 'r') as file:
    # Load the JSON data into a dictionary
    dict_seq_data = json.load(file)

# Print the dictionary
print(dict_seq_data)

{'2008': {'KANSAS': {'CHEYENNE': [[-101.9722961, 39.7901555], [-101.4333515, 39.4521148]], 'DECATUR': [[-100.7618523, 39.6677682], [-100.6932827, 39.6196749]], 'GRAHAM': [[-100.2632953, 39.7316189], [-99.5585669, 39.658828]], 'NORTON': [[-99.7577093, 39.9809822], [-100.2911213, 39.6423092]], 'RAWLINS': [[-101.2066149, 39.350858], [-101.079016, 39.8776223]], 'SHERIDAN': [[-100.136353, 39.0969773], [-100.8540012, 39.1568665]], 'SHERMAN': [[-101.7281407, 39.5211302], [-101.7201631, 39.2692347]], 'THOMAS': [[-100.7879757, 39.1579159], [-101.4323823, 39.5071463]], 'GOVE': [[-100.0441002, 38.9942628], [-100.6578149, 38.6073493]], 'GREELEY': [[-101.6928366, 38.3937351], [-101.364094, 38.6321247]], 'LANE': [[-100.8018296, 38.7268133], [-100.5558023, 38.4841847]], 'LOGAN': [[-100.9166677, 38.4292441], [-101.6114902, 38.7677492]], 'NESS': [[-100.4545828, 38.0449354], [-100.3443056, 38.1746594]], 'SCOTT': [[-100.9388214, 38.3520927], [-101.0164415, 38.6574763]], 'TREGO': [[-99.5788423, 39.1917965

### Convert json file to CSV file 

In [13]:
# Load json file and covert to CSV file
# Header: ['year', 'state_name', 'county_name', 'field_count']
import csv

working_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
output_seq_file = 'year_state_county_winterwheat_seq.csv'
# Open the CSV file in write mode
with open(working_dir+output_seq_file, 'w', newline='') as file:
    # Create a CSV writer object
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(['year', 'state_name', 'county_name', 'field_count'])
    
    # Iterate over the dictionary data
    for year, state_data in dict_seq_data.items():
        for state, county_data in state_data.items():
            for county, coordinates in county_data.items():
                # Calculate the count
                count = len(coordinates)
                
                # Write the data row
                writer.writerow([year, state, county, count])

# Print a confirmation message
print("CSV file created successfully!")


CSV file created successfully!


In [7]:
import pandas as pd

working_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'

df_cdl = pd.read_csv(working_dir + output_seq_file)
print(df_cdl)

      year state_name county_name  field_count
0     2008     KANSAS    CHEYENNE            2
1     2008     KANSAS     DECATUR            2
2     2008     KANSAS      GRAHAM            2
3     2008     KANSAS      NORTON            2
4     2008     KANSAS     RAWLINS            2
...    ...        ...         ...          ...
1570  2022     KANSAS     LABETTE            1
1571  2022     KANSAS  MONTGOMERY            1
1572  2022     KANSAS      NEOSHO            1
1573  2022     KANSAS      WILSON            0
1574  2022     KANSAS     WOODSON            0

[1575 rows x 4 columns]


In [11]:
# Load data set for Machine learning
working_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
# This csv file made by 'Fetching_Yield_AreaPlant_Data.ipynb' jupyter notebook
tgt_file = 'KENSAS_winter_wheat_yield_plant_data.csv'

df = pd.read_csv(working_dir + tgt_file)
print(df)

      year state_name county_name  yield  area_planted
0     2022     KANSAS       ALLEN  53.50         12100
1     2021     KANSAS       ALLEN  54.90         13800
2     2020     KANSAS       ALLEN  49.70             0
3     2019     KANSAS       ALLEN  44.50          4300
4     2018     KANSAS       ALLEN  43.65             0
...    ...        ...         ...    ...           ...
2096  2007     KANSAS   WYANDOTTE  20.00           600
2097  2006     KANSAS   WYANDOTTE  43.00           400
2098  2005     KANSAS   WYANDOTTE  43.00           400
2099  2004     KANSAS   WYANDOTTE  49.00           800
2100  2003     KANSAS   WYANDOTTE  63.00           400

[2101 rows x 5 columns]


### Add 'filed_count' column to original dataset

In [18]:
import np

merged_df = pd.merge(df, df_cdl, on=['year', 'county_name', 'state_name'], how='left')

# Rename the merged 'area_planted' column to avoid duplicates
merged_df.rename(columns={'field_count_y': 'field_count'}, inplace=True)
merged_df['field_count'] = merged_df['field_count'].astype(np.float64).fillna(0).astype(np.int64)

print(merged_df)

      year state_name county_name  yield  area_planted  field_count
0     2022     KANSAS       ALLEN  53.50         12100            0
1     2021     KANSAS       ALLEN  54.90         13800            2
2     2020     KANSAS       ALLEN  49.70             0            1
3     2019     KANSAS       ALLEN  44.50          4300            1
4     2018     KANSAS       ALLEN  43.65             0            0
...    ...        ...         ...    ...           ...          ...
2096  2007     KANSAS   WYANDOTTE  20.00           600            0
2097  2006     KANSAS   WYANDOTTE  43.00           400            0
2098  2005     KANSAS   WYANDOTTE  43.00           400            0
2099  2004     KANSAS   WYANDOTTE  49.00           800            0
2100  2003     KANSAS   WYANDOTTE  63.00           400            0

[2101 rows x 6 columns]


In [32]:
archives_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
tgt_file = 'KENSAS_winter_wheat_yield_plant_filedCnt_data.csv'

merged_df.to_csv(archives_dir + tgt_file, index=False)

In [25]:
# Check the total Winter Wheat plant area per year in Kansas state.
sum_by_year = df.groupby('year')['yield'].sum()
sum_by_year = sum_by_year.sort_values(ascending=False)
print(sum_by_year)

year
2003    5327.00
2016    5296.91
2021    5148.80
2017    5015.00
2019    4712.15
2020    4588.90
2012    4445.45
2018    4348.25
2022    4326.00
2010    4276.95
2013    4224.34
2009    4159.50
2005    4051.00
2015    4002.92
2004    3920.00
2008    3803.80
2011    3688.50
2006    3664.00
2014    3577.48
2007    3005.00
Name: yield, dtype: float64


### Cross-validate USDA-NASS Winter,wehat yield data with CDL data result

In [28]:
# Load data set for Machine learning
output_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/USDA-NASS--v01/'
# This csv file made by 'Fetching_Yield_AreaPlant_Data.ipynb' jupyter notebook
filename = 'repaired_yield__2023-06-06_15-26-38.csv'

df = pd.read_csv(output_dir + filename)

print(df)


     source_desc sector_desc   group_desc commodity_desc class_desc  \
0         SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
1         SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
2         SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
3         SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
4         SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
...          ...         ...          ...            ...        ...   
1812      SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
1813      SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
1814      SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
1815      SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   
1816      SURVEY       CROPS  FIELD CROPS          WHEAT     WINTER   

           prodn_practice_desc         util_practice_desc statisticcat_desc  \
0     ALL PRODUCTION PRACTICES  ALL UTILIZATION PRACTICES           

In [29]:
# Group the data by 'county' and get the minimum and maximum years for each county
county_years = df.groupby('county_name')['year'].agg(['min', 'max'])

start_year = 2003
end_year = 2022

# Iterate over each county and find missing years for each county
missing_years_by_county = {}
for county, years in county_years.iterrows():

    # Generate a list of all years between the start and end years for each county
    all_years = list(range(start_year, end_year + 1))

    # Find any missing years for each county by comparing the list of all years with the available years in the data
    missing_years_by_county[county] = list(set(all_years) - set(df[df['county_name'] == county]['year']))
    

print(missing_years_by_county)
print(len(missing_years_by_county))

{'ALLEN': [2016, 2017, 2018, 2020, 2010, 2012], 'ANDERSON': [2010, 2022], 'ATCHISON': [2016, 2012], 'BARBER': [2016, 2017, 2018, 2019, 2015], 'BARTON': [], 'BOURBON': [2019], 'BROWN': [2018], 'BUTLER': [2017], 'CHASE': [2013], 'CHAUTAUQUA': [2019, 2014], 'CHEROKEE': [2017, 2010], 'CHEYENNE': [2016], 'CLARK': [2014], 'CLAY': [2019, 2015], 'CLOUD': [2016, 2018, 2019, 2015], 'COFFEY': [2008, 2010, 2016, 2014], 'COMANCHE': [2012], 'COWLEY': [2017, 2018], 'CRAWFORD': [2018, 2013, 2014, 2015], 'DECATUR': [2018], 'DICKINSON': [2019], 'DONIPHAN': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015], 'DOUGLAS': [2016], 'EDWARDS': [2016], 'ELK': [2019, 2020, 2015], 'ELLIS': [2017, 2021], 'ELLSWORTH': [2016, 2013], 'FINNEY': [2016, 2017, 2018, 2019], 'FORD': [2019, 2021, 2022], 'FRANKLIN': [2008, 2013], 'GEARY': [2020], 'GOVE': [2017], 'GRAHAM': [2016, 2017, 2018, 2019, 2013, 2014, 2015], 'GRANT': [2008, 2018], 'GRAY': [2018, 2021], 'GREELEY': [2018], 'GREENW

In [33]:
import csv

archives_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
csv_file = 'KENSAS_winter_wheat_missedYear_data.csv'

# Define the column headers
fieldnames = ['county_name', 'year', 'is_omitted']

# Open the CSV file in write mode
with open(archives_dir+csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write the column headers to the CSV file
    writer.writeheader()

    # Write the dictionary data to the CSV file row by row
    for county_name, years in missing_years_by_county.items():
        for year in years:
            writer.writerow({'county_name': county_name, 'year': year, 'is_omitted': len(years)})            

In [38]:
archives_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
csv_file = 'KENSAS_winter_wheat_missedYear_data.csv'

df_missed_year = pd.read_csv(archives_dir + csv_file)

df_missed_year['is_omitted'] = 1
print(df_missed_year)

    county_name  year  is_omitted
0         ALLEN  2016           1
1         ALLEN  2017           1
2         ALLEN  2018           1
3         ALLEN  2020           1
4         ALLEN  2010           1
..          ...   ...         ...
278   WYANDOTTE  2011           1
279   WYANDOTTE  2012           1
280   WYANDOTTE  2013           1
281   WYANDOTTE  2014           1
282   WYANDOTTE  2015           1

[283 rows x 3 columns]


In [46]:
merged_df2 = pd.merge(merged_df, df_missed_year, on=['year', 'county_name'], how='left')

merged_df2['is_omitted'] = merged_df2['is_omitted'].astype(np.float64).fillna(0).astype(np.int64)

print(merged_df2)

archives_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
tgt_file = 'KENSAS_winter_wheat_yield_plant_filedCnt_omitChk_data.csv'

merged_df2.to_csv(archives_dir + tgt_file, index=False)

      year state_name county_name  yield  area_planted  field_count  \
0     2022     KANSAS       ALLEN  53.50         12100            0   
1     2021     KANSAS       ALLEN  54.90         13800            2   
2     2020     KANSAS       ALLEN  49.70             0            1   
3     2019     KANSAS       ALLEN  44.50          4300            1   
4     2018     KANSAS       ALLEN  43.65             0            0   
...    ...        ...         ...    ...           ...          ...   
2096  2007     KANSAS   WYANDOTTE  20.00           600            0   
2097  2006     KANSAS   WYANDOTTE  43.00           400            0   
2098  2005     KANSAS   WYANDOTTE  43.00           400            0   
2099  2004     KANSAS   WYANDOTTE  49.00           800            0   
2100  2003     KANSAS   WYANDOTTE  63.00           400            0   

      is_omitted  
0              0  
1              0  
2              1  
3              0  
4              1  
...          ...  
2096          

### Identify counties and years that are likely to not produce winter, wheat for real 

In [71]:
# Selected candidates 
# area_plnated = 0 && CDL white wheat filed count = 0 && no Yiled data at USDA-NASS && no Plnated area data at USDA-NASS

filtered_df = merged_df2[(merged_df2['area_planted'] == 0) & (merged_df2['field_count'] == 0) & (merged_df2['is_omitted'] == 1) & (merged_df2['yield'] > 0)]

print(len(filtered_df))
print(filtered_df)

archives_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
tgt_file = 'KENSAS_winter_wheat_yield_need_NDVI_chk_data.csv'

filtered_df.to_csv(archives_dir + tgt_file, index=False)

49
      year state_name   county_name  yield  area_planted  field_count  \
4     2018     KANSAS         ALLEN  43.65             0            0   
5     2017     KANSAS         ALLEN  42.80             0            0   
6     2016     KANSAS         ALLEN  41.95             0            0   
10    2012     KANSAS         ALLEN  40.95             0            0   
12    2010     KANSAS         ALLEN  33.15             0            0   
20    2022     KANSAS      ANDERSON  34.30             0            0   
32    2010     KANSAS      ANDERSON  35.60             0            0   
50    2012     KANSAS      ATCHISON  47.60             0            0   
103   2019     KANSAS       BOURBON  47.05             0            0   
124   2018     KANSAS         BROWN  33.20             0            0   
183   2019     KANSAS    CHAUTAUQUA  36.50             0            0   
205   2017     KANSAS      CHEROKEE  57.50             0            0   
212   2010     KANSAS      CHEROKEE  39.80      

#### Organize candidates' state name, county name, and year into a data frame 

In [68]:
filtered_df2 = filtered_df[['state_name', 'county_name', 'year']]
filtered_df2 = filtered_df2.drop_duplicates(subset=['state_name', 'county_name', 'year'])

print(filtered_df2)

     state_name   county_name  year
4        KANSAS         ALLEN  2018
5        KANSAS         ALLEN  2017
6        KANSAS         ALLEN  2016
10       KANSAS         ALLEN  2012
12       KANSAS         ALLEN  2010
20       KANSAS      ANDERSON  2022
32       KANSAS      ANDERSON  2010
50       KANSAS      ATCHISON  2012
103      KANSAS       BOURBON  2019
124      KANSAS         BROWN  2018
183      KANSAS    CHAUTAUQUA  2019
205      KANSAS      CHEROKEE  2017
212      KANSAS      CHEROKEE  2010
306      KANSAS        COFFEY  2016
312      KANSAS        COFFEY  2010
364      KANSAS      CRAWFORD  2018
369      KANSAS      CRAWFORD  2013
482      KANSAS           ELK  2020
483      KANSAS           ELK  2019
594      KANSAS      FRANKLIN  2008
730      KANSAS     GREENWOOD  2012
732      KANSAS     GREENWOOD  2010
734      KANSAS     GREENWOOD  2008
863      KANSAS     JEFFERSON  2019
868      KANSAS     JEFFERSON  2014
900      KANSAS       JOHNSON  2022
904      KANSAS       JOHNSO

#### Add lon and lat information to verify NDVI information 

In [67]:
import pandas as pd

archive_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
scll_file = 'state_county_lon_lat.csv'

df_scll = pd.read_csv(archive_dir + scll_file)

print(df_scll)


    state_name county_name         lon        lat
0       KANSAS    CHEYENNE -101.757549  39.795580
1       KANSAS     DECATUR -100.472769  39.794053
2       KANSAS      GRAHAM  -99.898062  39.340620
3       KANSAS      NORTON  -99.910003  39.794470
4       KANSAS     RAWLINS -101.099472  39.790480
..         ...         ...         ...        ...
100     KANSAS     LABETTE  -95.310373  37.189971
101     KANSAS  MONTGOMERY  -95.762834  37.198395
102     KANSAS      NEOSHO  -95.331660  37.556635
103     KANSAS      WILSON  -95.745264  37.569212
104     KANSAS     WOODSON  -95.751711  37.881137

[105 rows x 4 columns]


In [69]:
df_yscyll = pd.merge(filtered_df2,df_scll, on=['state_name','county_name'],how='left')

print(df_yscyll)

   state_name   county_name  year        lon        lat
0      KANSAS         ALLEN  2018 -95.317021  37.888217
1      KANSAS         ALLEN  2017 -95.317021  37.888217
2      KANSAS         ALLEN  2016 -95.317021  37.888217
3      KANSAS         ALLEN  2012 -95.317021  37.888217
4      KANSAS         ALLEN  2010 -95.317021  37.888217
5      KANSAS      ANDERSON  2022 -95.306746  38.198077
6      KANSAS      ANDERSON  2010 -95.306746  38.198077
7      KANSAS      ATCHISON  2012 -95.332605  39.545816
8      KANSAS       BOURBON  2019 -94.867959  37.848580
9      KANSAS         BROWN  2018 -95.581995  39.824223
10     KANSAS    CHAUTAUQUA  2019 -96.265697  37.146335
11     KANSAS      CHEROKEE  2017 -94.848207  37.171807
12     KANSAS      CHEROKEE  2010 -94.848207  37.171807
13     KANSAS        COFFEY  2016 -95.749893  38.235413
14     KANSAS        COFFEY  2010 -95.749893  38.235413
15     KANSAS      CRAWFORD  2018 -94.865247  37.503889
16     KANSAS      CRAWFORD  2013 -94.865247  37

In [70]:
archive_dir = '/Users/jinholee/Desktop_local/2023_Spring_FoodSecurity/HW3/output/ML-ARCHIVES--v01/'
yscyll_filename = 'KENSAS_winter_wheat_yield_need_NDVI_chk_lon_lat.csv'
df_yscyll.to_csv(archive_dir + yscyll_filename, index=False)